## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. They often do that by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract BGP configuration and session status information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [45]:
# Import packages and load questions
%run startup.py

 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [46]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example-bgp"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

load_questions()

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### View BGP Configuration for ALL devices
Batfish makes BGP Configuration settings in the network easily accessible. Let's take a look at how you can retrieve the specific information you want. Let's start with configuration attributes of the BGP process on all devices running BGP.

In [47]:
# Get BGP process configuration information for ALL devices
bgp_config = bfq.bgpProcessConfiguration().answer().frame()
bgp_config

,Node,VRF,Router_ID,Route_Reflector,Multipath_Match_Mode,Multipath_EBGP,Multipath_IBGP,Neighbors,Tie_Breaker
0,as1border1,default,1.1.1.1,False,EXACT_PATH,True,True,"['1.10.1.1/32', '3.2.2.2/32', '5.6.7.8/32', '10.12.11.2/32']",ARRIVAL_ORDER
1,as2dist2,default,2.1.3.2,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",ARRIVAL_ORDER
2,as2border1,default,2.1.1.1,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '10.12.11.1/32']",ARRIVAL_ORDER
3,as2core2,default,2.1.2.2,True,EXACT_PATH,True,True,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",ARRIVAL_ORDER
4,as3border1,default,3.1.1.1,False,EXACT_PATH,True,True,"['3.10.1.1/32', '10.23.21.2/32']",ARRIVAL_ORDER
5,as3border2,default,3.2.2.2,False,EXACT_PATH,True,True,"['3.10.1.1/32', '10.13.22.1/32']",ARRIVAL_ORDER
6,as3core1,default,3.10.1.1,True,EXACT_PATH,True,True,"['3.1.1.1/32', '3.2.2.2/32']",ARRIVAL_ORDER
7,as2dist1,default,2.1.3.1,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",ARRIVAL_ORDER
8,as1border2,default,1.2.2.2,False,EXACT_PATH,True,True,"['1.10.1.1/32', '10.13.22.3/32', '10.14.22.4/32']",ARRIVAL_ORDER
9,as2border2,default,2.1.1.2,False,EXACT_PATH,True,True,"['2.1.2.1/32', '2.1.2.2/32', '10.23.21.3/32']",ARRIVAL_ORDER


NOTE: Any router that has a `Cluster ID` set is configured to be a route-reflector for some of it's BGP peers

In [48]:
# wrapper function that will return the definition of a routing policy
#
# Input:
#    node: Name of the device
#    policy_names: List of policy names to retrieve
#    config_file_path: Directory where the `configs` folder resides
#
# Output:
#    policy_def: dictionary keyed on policy name that contains the actual config lines that define the policy
def get_bgp_policies(node, policy_names, config_file_path):
    
    import linecache
    from collections import defaultdict
    
    # This returns the list of all defined structures on `as2dept1`
    def_struct = bfq.definedStructures(nodes=node).answer().frame()
    
    policy_def = defaultdict(list)

    # Retrieving just the structures that are used as BGP import policies for the specific peer we are looking at

    for policy in policy_names:
        struct = def_struct[def_struct['Structure_Name']==policy]
        file = config_file_path + "/" + struct.Source_Lines.iloc[0].filename

        lines = struct.Source_Lines.iloc[0].lines

        text_line = []

        for line in lines:
            text_line.append(linecache.getline(file, line))
            
        policy_def[policy] = text_line
            
    return(policy_def)

Now let's drill into the configuration of a specific BGP session. Let's look at the sessions on `as2dept1`

In [49]:
# Get all of the BGP peer configuration for ALl devices
bgp_peer_config = bfq.bgpPeerConfiguration().answer().frame()
bgp_peer_config[bgp_peer_config['Node'].str.contains('as2dept1')]

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
4,as2dept1,default,65001,2.34.101.4,2,2.34.101.3,False,None,as2,['as2_to_dept'],['dept_to_as2'],True,False
36,as2dept1,default,65001,2.34.201.4,2,2.34.201.3,False,None,as2,['as2_to_dept'],['dept_to_as2'],True,False


`as2dept1` has 2 configured bgp peers. Let's take a look at the input and output route policies for the peer `2.24.101.3`

In [50]:
# as2dept1 BGP peer configuration that we want to dig into
peer_ip = '2.34.101.3'

peer_import_policy = bgp_peer_config[(bgp_peer_config['Node'].str.contains('as2dept1')) & (bgp_peer_config['Remote_IP']==peer_ip)].Import_Policy.iloc[0]
peer_export_policy = bgp_peer_config[(bgp_peer_config['Node'].str.contains('as2dept1')) & (bgp_peer_config['Remote_IP']==peer_ip)].Export_Policy.iloc[0]

In [51]:
import_policy_text = get_bgp_policies('as2dept1', peer_import_policy, SNAPSHOT_PATH)

for import_policy in import_policy_text:
    print("Import Policy: {}".format(import_policy))
    print("Definition: {}".format(import_policy_text[import_policy]))

Import Policy: as2_to_dept
Definition: ['route-map as2_to_dept permit 100\n', ' match community as2_community\n', ' set local-preference 350\n']


In [52]:
export_policy_text = get_bgp_policies('as2dept1', peer_export_policy, SNAPSHOT_PATH)

for export_policy in export_policy_text:
    print("Export Policy: {}".format(export_policy))
    print("Definition: {}".format(export_policy_text[export_policy]))

Export Policy: dept_to_as2
Definition: ['route-map dept_to_as2 permit 100\n', ' match ip address 102\n', ' set metric 50\n', ' set community 65001:2 additive\n']


### View BGP Session Status

Now that we have seen the configuration for each peer on `as2dept1`, let's ensure that their configuration is compatible with their peers

In [53]:
# The bgpSessionCompatibility question allows you to ensure that BGP sessions are compatibly configured, so that if there is IP reachability between the peers
# the sessions will be established. Compatiblity checks that the remote-as matches up on both ends, the correct update source is specified, etc...

bgpSessCompat = bfq.bgpSessionCompatibility(nodes='as2dept1').answer().frame()
bgpSessCompat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
0,as2dept1,default,65001,as2dept1:GigabitEthernet0/0,2.34.101.4,2,as2dist1,2.34.101.3,EBGP_SINGLEHOP,UNIQUE_MATCH
1,as2dept1,default,65001,as2dept1:GigabitEthernet1/0,2.34.201.4,2,as2dist2,2.34.201.3,EBGP_SINGLEHOP,UNIQUE_MATCH


Both of the configured BGP peers on `as2dept1` are compatible. We know that since the `Configured_Status` is `UNIQUE_MATCH`. So now let's check if they are established.

In [54]:
bgpSessStat = bfq.bgpSessionStatus(nodes='as2dept1').answer().frame()
bgpSessStat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
0,as2dept1,default,65001,as2dept1:GigabitEthernet0/0,2.34.101.4,2,as2dist1,2.34.101.3,EBGP_SINGLEHOP,ESTABLISHED
1,as2dept1,default,65001,as2dept1:GigabitEthernet1/0,2.34.201.4,2,as2dist2,2.34.201.3,EBGP_SINGLEHOP,ESTABLISHED


Both sessions are established. Let's see if there are any configured BGP sessions, on any other device in the network, that are not established

In [55]:
bgpSessStat = bfq.bgpSessionStatus().answer().frame()
bgpSessStat[bgpSessStat['Established_Status'] != 'ESTABLISHED']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
1,as1border1,default,1,None,None,666,None,3.2.2.2,EBGP_SINGLEHOP,NOT_COMPATIBLE
2,as1border1,default,1,None,None,555,None,5.6.7.8,EBGP_SINGLEHOP,NOT_COMPATIBLE
6,as1border2,default,1,as1border2:GigabitEthernet2/0,10.14.22.1,4,None,10.14.22.4,EBGP_SINGLEHOP,NOT_COMPATIBLE
9,as2border1,default,2,as2border1:Loopback0,2.1.1.1,2,as2core1,2.1.2.1,IBGP,NOT_ESTABLISHED
12,as2border2,default,2,as2border2:Loopback0,2.1.1.2,2,as2core1,2.1.2.1,IBGP,NOT_ESTABLISHED
13,as2border2,default,2,None,None,2,None,2.1.2.2,IBGP,NOT_COMPATIBLE
15,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2border1,2.1.1.1,IBGP,NOT_ESTABLISHED
16,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2border2,2.1.1.2,IBGP,NOT_ESTABLISHED
17,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist1,2.1.3.1,IBGP,NOT_ESTABLISHED
18,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist2,2.1.3.2,IBGP,NOT_ESTABLISHED


Wow, there are a lot of sessions that are configured, but not established. Let's dig into these issues. Let's find all of the sessions that are not compatibly configured. For that we are looking for sessions which are not either a `UNIQUE_MATCH` or `DYNAMIC_MATCH`. The latter is for peers that are configured with a `listen-range`

In [56]:
bgpSessCompat = bfq.bgpSessionCompatibility().answer().frame()
bgpSessCompat[~bgpSessCompat['Configured_Status'].isin(["UNIQUE_MATCH", "DYNAMIC_MATCH"])]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
1,as1border1,default,1,None,None,666,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP
6,as1border2,default,1,as1border2:GigabitEthernet2/0,10.14.22.1,4,None,10.14.22.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
13,as2border2,default,2,None,None,2,None,2.1.2.2,IBGP,LOCAL_IP_UNKNOWN_STATICALLY
20,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,None,2.1.1.2,IBGP,HALF_OPEN


Let's dig into the sessions that are `LOCAL_IP_UNKNOWN_STATICALLY` and `HALF_OPEN`. 

An iBGP session will have the status `LOCAL_IP_UNKNOWN_STATICALLY` if you are missing the `update-source` command.

A BGP session will be `HALF_OPEN` when Batfish is able to find both peers, but cannot fully evaluate the compatibility. This will happen if only one of the 2 routers that are forming a session via iBGP have the `update-source` command configured.

So, in this case, it is likely that the issue is that `as2border2` is missing the `update-source` command on it's BGP session to `as2core2`. We can check this by looking at the `bfq.ipOwners` output

In [66]:
ipOwn = bfq.ipOwners().answer().frame()
ipOwn[ipOwn['IP'].isin(['2.1.1.2', '2.1.2.2'])]

,Node,VRF,Interface,IP,Mask,Active
22,as2core2,default,Loopback0,2.1.2.2,32,True
37,as2border2,default,Loopback0,2.1.1.2,32,True


Let's check if the session actually gets established, based on our theory


In [74]:
# check if either direction of the session can be established
bgpSessStat[((bgpSessStat['Local_IP']=='2.1.1.2') & (bgpSessStat['Remote_IP']=='2.1.2.2')) | ((bgpSessStat['Local_IP']=='2.1.2.2') & (bgpSessStat['Remote_IP']=='2.1.1.2'))]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
20,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,None,2.1.1.2,IBGP,NOT_COMPATIBLE


In [76]:
# check if there is a node in the network that has the `10.14.22.4` on an interface
ipOwn[ipOwn['IP']=='10.14.22.4']

,Node,VRF,Interface,IP,Mask,Active


Batfish deems the BGP peer `10.14.22.4` is not compatible because it cannot find a device in the snapshot that has that IP address configured on any interface. That is why the status is `UNKNOWN_REMOTE`.
This will occur in  most networks, since you will not have the configurations of the devices for your external peers (ISPs, content partners, etc...)

Let's dig into the 2 peers on `as1border1`, which have status `NO_LOCAL_IP`next.

In [77]:
#BGP Peer configuration for `as1border1`
bgp_peer_config[(bgp_peer_config['Node']=='as1border1') & (bgp_peer_config['Local_IP'].isnull())]

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
16,as1border1,default,1,None,666,3.2.2.2,False,None,bad-ebgp,[],[],False,False
23,as1border1,default,1,None,555,5.6.7.8,False,None,xanadu,[],[],False,False


Batfish determines the Local_IP for each BGP session, either based on explicit configuration with `updates-source foo` (or equivalent non-IOS command) for iBGP sessions or eBGP multi-hop sessions,
or by determining the interface the router will use to send packets towards the BGP peer. The latter method requires the route to the peer to be known.

So, if there is no route to the configured peer, or the configured peer does not exist in the snapshot, you will see this status. We can check the output of `bfq.routes` and `bfq.ipOwners` questions to dig into this

In [78]:
bad_bgp_peer = ['3.2.2.2', '5.6.7.8']

In [79]:
ipOwn[ipOwn['IP'].isin(bad_bgp_peer)]

,Node,VRF,Interface,IP,Mask,Active
10,as3border2,default,Loopback0,3.2.2.2,32,True


So we can see that `5.6.7.8` does not exist in the network. This either means there is a mis-configuration, or the device is just not expected to be in the snapshot.

Now let's dig into the peer `3.2.2.2`, which we know is the Loopback interface on `as3border2`

In [80]:
routes = bfq.routes(network='3.2.2.2/32').answer().frame()
routes[routes['Node']=='as1border1']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Protocol,Metric,Admin_Distance,Tag


The specific /32 is not present on `as1border1`. What about other routers?

In [81]:
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Protocol,Metric,Admin_Distance,Tag
0,as3border1,default,3.2.2.2/32,as3core1,3.0.1.2,ospf,3,110,None
1,as3core1,default,3.2.2.2/32,as3border2,3.0.2.1,ospf,2,110,None
2,as3border2,default,3.2.2.2/32,None,AUTO/NONE(-1l),connected,0,0,None


We can see that this route does not leave `as3`, which is why `as1border1` is unable to established the configured BGP session.
This session should probably have been configured as an eBGP multi-hop session with static routes pointing to the appropriate interface and next-hop

### Now let's look at how Batfish treats dynamic BGP sessions configured using a `listen-range`

We identify routers that are configured with dynamic BGP sessions by looking for `DYNAMIC_MATCH` in the `bfq.bgpSessionCompatibility` question

In [83]:
bgpSessCompat[bgpSessCompat['Configured_Status']=='DYNAMIC_MATCH']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
35,as2dist1,default,2,as2dist1:GigabitEthernet2/0,2.34.101.3,65001,as2dept1,2.34.101.4,EBGP_SINGLEHOP,DYNAMIC_MATCH
36,as2dist2,default,2,as2dist2:GigabitEthernet2/0,2.34.201.3,65001,as2dept1,2.34.201.4,EBGP_SINGLEHOP,DYNAMIC_MATCH


Now let's check the status of these BGP sessions. Let's look at the session between `2.34.101.3` and `2.34.101.4`

In [85]:
bgpSessStat[((bgpSessStat['Local_IP']=='2.34.101.3') & (bgpSessStat['Remote_IP']=='2.34.101.4')) | ((bgpSessStat['Local_IP']=='2.34.101.4') & (bgpSessStat['Remote_IP']=='2.34.101.3'))]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
23,as2dept1,default,65001,as2dept1:GigabitEthernet0/0,2.34.101.4,2,as2dist1,2.34.101.3,EBGP_SINGLEHOP,ESTABLISHED
35,as2dist1,default,2,as2dist1:GigabitEthernet2/0,2.34.101.3,65001,as2dept1,2.34.101.4,EBGP_SINGLEHOP,ESTABLISHED


Let's look at the session between `2.34.201.3` and `2.34.201.4`

In [86]:
bgpSessStat[((bgpSessStat['Local_IP']=='2.34.201.3') & (bgpSessStat['Remote_IP']=='2.34.201.4')) | ((bgpSessStat['Local_IP']=='2.34.201.4') & (bgpSessStat['Remote_IP']=='2.34.201.3'))]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
24,as2dept1,default,65001,as2dept1:GigabitEthernet1/0,2.34.201.4,2,as2dist2,2.34.201.3,EBGP_SINGLEHOP,ESTABLISHED
36,as2dist2,default,2,as2dist2:GigabitEthernet2/0,2.34.201.3,65001,as2dept1,2.34.201.4,EBGP_SINGLEHOP,ESTABLISHED


As you can see, both dynamic BGP sessions were established.

We hope you found this notebook useful and informative. With Batfish you can easily retrieve information about BGP configuration of all devices and peers, as well as status of each peer.

Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.